In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import json
import re
import numpy as np

In [2]:
def load_json(file_path, dataset):
    with open(file_path, 'r') as file:
        for line in file:
            dataset.append(json.loads(line))
    return dataset

def remove_columns(dataset, numeric_columns, categorical_columns):
    for column in dataset.columns:
        if column not in numeric_columns and column not in categorical_columns and column != 'label':
            dataset.drop(column, axis=1, inplace=True)
    return dataset

def replace_null_values(dataset, numeric_columns, categorical_columns,numeric_imputer, categorical_imputer):
    dataset[numeric_columns] = numeric_imputer.fit_transform(dataset[numeric_columns])
    dataset[categorical_columns] = categorical_imputer.fit_transform(dataset[categorical_columns])
    return dataset

def normalize_numeric_columns(dataset, numeric_columns):
    scaler = MinMaxScaler()
    dataset[numeric_columns] = scaler.fit_transform(dataset[numeric_columns])
    return dataset

def top_n(dataset, categorical_columns):
    top_val = 10
    for col in categorical_columns:
        value_counts = dataset[col].value_counts()
        top_n_categories = value_counts.index[:top_val].tolist()
        dataset[col] = dataset[col].where(dataset[col].isin(top_n_categories), other='Other')
    return dataset

def align_columns(train, test):
   
    missing_in_test = set(train.columns) - set(test.columns)
    missing_in_train = set(test.columns) - set(train.columns) 
  
    for col in missing_in_test: # aggiungi le colonne mancanti in test e riempi con False
        test[col] = False
    
    for col in missing_in_train: # aggiungi le colonne mancanti in train e riempi con False
        train[col] = False

    # riordina le colonne in entrambi i dataset per avere la stessa struttura
    train = train.reindex(sorted(train.columns), axis=1)
    test = test.reindex(sorted(test.columns), axis=1)

    return train, test

In [3]:
dataset=pd.read_csv('dati/dataset_train.csv')
dataset_test=pd.read_csv('dati/dataset_test.csv')

In [ ]:
#features

#dest_ip: L'indirizzo IP di destinazione
#
#epochdate: Un timestamp in formato epoch
#
#source_ip: L'indirizzo IP di origine
#
#@host: L'host che ha registrato l'evento
#
#@timestamp: La data e l'ora in cui è stato registrato l'evento (ad esempio, "2021-10-04T00:00:00.000Z")
#
#dest_port: La porta di destinazione utilizzata per la connessione SSH. Tipicamente è la porta 22 (default per SSH), ma potrebbe essere personalizzata.
#
#status_guess: Un'indicazione dello stato della connessione (ad esempio, successo o fallimento)
#
#uid: Un identificatore univoco per la sessione o per la connessione SSH
#
#@type: Il tipo di evento o di log
# 
#source_port: La porta di origine utilizzata per la connessione
#
#host_key_alg: L'algoritmo utilizzato per la chiave host del server SSH
#
#host_key: La chiave pubblica del server SSH. Viene utilizzato dal client per verificare l'autenticità del server durante la negoziazione della connessione
#
#server: Nome o identificativo del server che gestisce la connessione SSH.
#
#version: La versione del protocollo SSH utilizzata per la connessione 
#
#remote_location_latitude: La latitudine della posizione geografica dell'indirizzo IP di origine
# 
#compression_alg: L'algoritmo di compressione utilizzato (se presente) nella sessione SSH per ridurre il traffico trasmesso
#
#kex_alg: L'algoritmo di scambio delle chiavi utilizzato nella negoziazione della sessione SSH
#
#remote_location_longitude: La longitudine della posizione geografica dell'indirizzo IP di origine
#
#auth_attempts: Il numero di tentativi di autenticazione fatti dal client per stabilire la connessione SSH
#
#remote_location_country_code: Il codice del paese di origine dell'indirizzo IP
#
#cipher_alg: L'algoritmo di cifratura utilizzato per la sessione SSH
#
#mac_alg: L'algoritmo per il Message Authentication Code (MAC), utilizzato per verificare l'integrità dei dati nella sessione SS
#
#auth_success: Un booleano che indica se il tentativo di autenticazione è stato un successo o meno
#
#direction: La direzione della connessione SSH. Può indicare se la connessione è stata "inbound" (in arrivo verso il server) o "outbound" (in uscita dal server)
#
#client: Il client SSH utilizzato per stabilire la connessione


In [4]:
numeric_columns = [ 'auth_success','auth_attempts']
categorical_columns=['dest_port', 'status_guess', 'version','kex_alg','mac_alg','host_key_alg', 'cipher_alg', 'client', 'direction']

dataset=remove_columns(dataset, numeric_columns, categorical_columns)
dataset_test=remove_columns(dataset_test, numeric_columns, categorical_columns)


In [5]:
numeric_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')

dataset = replace_null_values(dataset, numeric_columns, categorical_columns, numeric_imputer, categorical_imputer)
dataset = normalize_numeric_columns(dataset, numeric_columns)

dataset_test = replace_null_values(dataset_test, numeric_columns, categorical_columns, numeric_imputer, categorical_imputer)
dataset_test = normalize_numeric_columns(dataset_test, numeric_columns)


In [6]:
print(dataset.shape)
print(dataset_test.shape)

(433929, 12)
(132703, 12)


In [7]:
dataset = top_n(dataset, categorical_columns)
dataset_test = top_n(dataset_test, categorical_columns)

# Codifica dei dati categorici con get_dummies di pandas per la codifica one-hot
categorical_encoded = pd.get_dummies(dataset[categorical_columns])
categorical_encoded_test = pd.get_dummies(dataset_test[categorical_columns])

dataset = dataset.drop(columns=categorical_columns, axis=1) # Rimozione delle colonne categoriche originali
dataset = pd.concat([dataset, categorical_encoded], axis=1) # Concatenazione delle nuove colonne codificate


dataset_test = dataset_test.drop(columns=categorical_columns, axis=1)  # Rimozione delle colonne categoriche originali 
dataset_test = pd.concat([dataset_test, categorical_encoded_test], axis=1) # Concatenazione delle nuove colonne codificate


In [8]:
print(dataset.shape)
print(dataset_test.shape)

(433929, 64)
(132703, 60)


In [ ]:
dataset, dataset_test = align_columns(dataset, dataset_test)

print(dataset_test.head())
print(dataset.head())
print(dataset.columns.to_list())
print(dataset_test.columns.to_list())

In [10]:
print(dataset.shape)
print(dataset_test.shape)

(433929, 68)
(132703, 68)


In [14]:
#ESEMPIO FUNZIONAMENTO align_columns
# Dataset di train (ha le colonne 'A', 'B', 'C')
data_train = {'A': [1, 2, 3],
              'B': [4, 5, 6],
              'C': [7, 8, 9]}

train = pd.DataFrame(data_train)

# Dataset di test (ha le colonne 'B', 'C', 'D')
data_test = {'B': [10, 11, 12],
             'C': [13, 14, 15],
             'D': [16, 17, 18]}

test = pd.DataFrame(data_test)

print("\ntrain:")
print(train)
print("\ntest:")
print(test)

train, test = align_columns(train, test)

print("\ntrain modificato:")
print(train)
print("\ntest modificato:")
print(test)


train:
   A  B  C
0  1  4  7
1  2  5  8
2  3  6  9

test:
    B   C   D
0  10  13  16
1  11  14  17
2  12  15  18

train modificato:
   A  B  C      D
0  1  4  7  False
1  2  5  8  False
2  3  6  9  False

test modificato:
       A   B   C   D
0  False  10  13  16
1  False  11  14  17
2  False  12  15  18


In [ ]:
print("\nAnteprima dopo la codifica one-hot:")
print(dataset.head())
print(dataset_test.head())

In [12]:
data_label_0 = dataset[dataset['label'] == 0]
data_label_1 = dataset[dataset['label'] != 0]
X_train=data_label_0.drop('label', axis=1)


In [13]:
y_test = dataset_test['label']
X_test = dataset_test.drop('label', axis=1)


In [14]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')


In [ ]:
X_train_df = pd.DataFrame(X_train)
X_train_df.to_csv('dati/X_train.csv', index=False)
print(X_train_df.head())
X_test_df = pd.DataFrame(X_test)
X_test_df.to_csv('dati/X_test.csv', index=False)
print(X_test_df.head())
y_test_df = pd.DataFrame(y_test)
y_test_df.to_csv('dati/y_test.csv', index=False)
print(y_test_df.head())

In [ ]:
print(X_test.columns.to_list())
print(X_train.columns.to_list())